In [ ]:
import pandas as pd
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.display import display
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
data = pd.read_csv('data.csv')
data

In [ ]:
nombre_ana =  data[data["nombre"] == "Ana"]
nombre_ana

In [ ]:
columnas = data.columns.to_list()
columnas

In [ ]:
def parametro_busqueda_keyword(parametro):
    res =  data[parametro]
    print(res)


In [ ]:


def user_input(debug):

    if debug:
        return "muestrame algunos nombres"
    else:
        text_input = widgets.Text(
            value='',
            description='Tu pregunta:',
            placeholder='Escribe tu pregunta aquí'
        )
        
        display(text_input)
        
        def handle_submit(sender):
            global user_question
            user_question = sender.value
            clear_output()
            print(f"Pregunta ingresada: {user_question}")
        
        text_input.observe(handle_submit, names='value')
        
        return text_input.value

In [ ]:

checkpoint = "bigscience/bloomz-1b7"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)


In [ ]:
prompt_2 = f"""
    INSTRUCCIÓN: Elige UNA palabra de esta lista [{', '.join(columnas)}] que mejor clasifique la siguiente pregunta
    PREGUNTA: 'Quien tiene 25 años?'.
    IMPORTANTE: Solo responde con UNA PALABRA DE LA LISTA. Nada más.
"""


inputs_2 = tokenizer.encode(prompt_2, return_tensors="pt")
attention_mask = torch.ones_like(inputs_2)

outputs = model.generate(
    inputs_2,
    attention_mask=attention_mask,
    max_new_tokens=2,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.7,
    top_k=5,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.2
)

res = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
print(res)

In [ ]:
prompt = f"""INSTRUCCIÓN: Elige UNA palabra de esta lista [{', '.join(columnas)}] que mejor clasifique la siguiente frase.
FRASE: '{user_input(True)}'
IMPORTANTE: Solo responde con una palabra de la lista. Nada más.
RESPUESTA:"""

inputs = tokenizer.encode(prompt, return_tensors="pt")
attention_mask = torch.ones_like(inputs)

outputs = model.generate(
    inputs,
    attention_mask=attention_mask,
    max_new_tokens=2,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.7,
    top_k=5,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.2
)

res = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
words = res.split()
for word in reversed(words):
    word = word.strip("'\".,!? ")
    if word in columnas:
        last_word = word
        break
    else:
        last_word = "nombre"

print(f"Palabra clasificada: {last_word}")
parametro_busqueda_keyword(str(last_word))

In [ ]:
import torch
from transformers import pipeline
from dotenv import load_dotenv
import os

load_dotenv()

model_id = "meta-llama/Llama-3.2-1B"

pipe = pipeline(
    "text-generation", 
    model=model_id, 
    token=os.getenv("HF_LLAMA_TOKEN"),
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

pipe("The key to life is")


Some parameters are on the meta device because they were offloaded to the disk and cpu.
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'The key to life is not to lose yourself in the world but to find yourself in the world.\nThe key to life is not to lose yourself in the world but to find yourself in the world. This is the key to life. The key to life is not to lose yourself in the world but to find yourself in the world. This is the key to life. This is the key to life.\nThe key to life is not to lose yourself in the world but to find yourself in the world.\nThe key to life is not to lose yourself in the world but to find yourself in the world.\nThe key to life is not to lose yourself in the world but to find yourself in the world. This is the key to life. The key to life is not to lose yourself in the world but to find yourself in the world. This is the key to life. This is the key to life.\nThe key to life is not to lose yourself in the world but to find yourself in the world.\nThe key to life is not to lose yourself in the world but to find yourself in the world. This is the key to life. The ke